<img src="tic3151.png" style="height: 70px; width: 200px" align=left> 
<img src="logo-MMU.png" style="height: 70px; width: 200px" align=right> 

### Team members

Ahmad Ayaan - 1191302794 

Salah Fayeq y. AlHaismawi - 1201303035

Aisha Sakinah Binti Saadon - 1191101867

Aryssa Amanda Binti Azhar - 1191100575


### The assignment contains following questions


# Installing Libraries

##### Note: Uncomment the code the code to install the relevant libraries

In [1]:
# pip install pygad


# Importing Libraries

In [2]:
from random import randint
import random


# Question 1

Year-end school holiday is coming soon. A kindergarten teacher would like to give each
of the student in her class, an amazing holiday set by November 2023. There are 25
students in her class consisting of 15 girls and 10 boys. She has approached you for
help and provided the following details to you:

• Budget for 25 holiday sets = RM 200.00

• Each holiday set must be wrapped using a sheet of paper i.e. blue wrapper for
boy’s holiday set and pink wrapper for girl’s holiday set. Each wrapper cost
RM1.00

• Each holiday set should have at least 2 items being stationery



## Variables Declaration

In [3]:
# Constant variables
budget = 200.00
minimum_stationary_item = 2
wrapper_cost = 1.00

# List of items and their prices
items = [
    ("Chipsmore Biscuit", 2.00),
    ("2B pencil", 1.00),
    ("Color pencil", 3.00),
    ("Magic pen", 3.00),
    ("Potato chip", 2.00),
    ("Milo packet", 1.00),
    ("Mini sketch book", 2.00),
    ("Milk biscuit", 3.00)
]

## Population and Individuals



In [4]:
def individuals():
    return [random.randint(0, 1) for items in range(len(items))]

def population(population_size):
    return [individuals() for x in range(population_size)]

## Fitness Function

The fitness function calculates the total cost of items in a chromosome, after deducting the cost of wrappers and ensuring that atleast 2 of the items are stationery.

In [5]:
def fitness_function(chromosome):
    total = 0
    num_stationery_items = 0
    for i, j in enumerate(chromosome):
        if j == 1:
            # to calculate the total cost of the items present in the set
            total += items[i][1]
            # to check if the current sets have stationery
            if items[i][0] in ["2B pencil", "Color pencil", "Magic pen", "Mini sketch book"]:
                num_stationery_items += 1

    if num_stationery_items >= 2:
        return total
    else:
        return 0

## Selection Function

In [6]:
"""Perform selection using roulette wheel selection."""
def selection_roulette_wheel(population, fitness_values):
    total_fitness = sum(fitness_values)
    probabilities = [fitness / total_fitness for fitness in fitness_values]
    selected = random.choices(population, weights=probabilities, k=2)
    return selected[0], selected[1]

"""Perform selection using rank selection."""
def selection_rank(population, fitness_values):
    ranked_population = sorted(zip(population, fitness_values), key=lambda x: x[1], reverse=True)
    ranked_probs = [2 * (i + 1) / (len(population) * (len(population) + 1)) for i in range(len(population))]
    selected = random.choices(population, weights=ranked_probs, k=2)
    return selected[0], selected[1]

## Crossover Function

In [7]:
"""Perform crossover using single point crossover."""
def crossover_single_point(parent1, parent2):
    single_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:single_point] + parent2[single_point:]
    child2 = parent2[:single_point] + parent1[single_point:]
    return child1, child2

"""Perform crossover using two point crossover."""
def crossover_two_point(parent1, parent2):
    two_point = sorted(random.sample(range(len(parent1)), 2))
    child1 = parent1[:two_point[0]] + parent2[two_point[0]:two_point[1]] + parent1[two_point[1]:]
    child2 = parent2[:two_point[0]] + parent1[two_point[0]:two_point[1]] + parent2[two_point[1]:]
    return child1, child2

## Mutation Function

In [8]:
"""Perform inversion mutation."""
def mutation_inversion(chromosome):
    start = random.randint(0, len(chromosome) - 1)
    end = random.randint(start + 1, len(chromosome))
    mutated_chromosome = chromosome[:start] + chromosome[start:end][::-1] + chromosome[end:]
    return mutated_chromosome

"""Perform bit flipping mutation."""
def mutation_bit_flip(chromosome, mutation_rate):
    mutated_chromosome = chromosome[:]
    for i in range(len(mutated_chromosome)):
        if random.random() < mutation_rate:
            mutated_chromosome[i] = 1 - mutated_chromosome[i]
    return mutated_chromosome

## Main Function

In [9]:
#these are dummy main functions to test each stages of development

def to_check_fitness():
    pop = population(25) 

    # Calculate fitness values for each chromosome
    fitness_values = [fitness_function(chromosome) for chromosome in pop]
    # Print the fitness values
    for chromosome, fitness in zip(pop, fitness_values):
        print("Chromosome:", chromosome)
        print("Fitness Value:", fitness)  
    
to_check_fitness()

Chromosome: [0, 0, 1, 0, 1, 1, 0, 1]
Fitness Value: 0
Chromosome: [0, 0, 0, 0, 1, 1, 1, 1]
Fitness Value: 0
Chromosome: [0, 1, 1, 1, 0, 1, 0, 1]
Fitness Value: 11.0
Chromosome: [0, 0, 0, 1, 0, 0, 0, 0]
Fitness Value: 0
Chromosome: [0, 0, 1, 1, 0, 1, 0, 0]
Fitness Value: 7.0
Chromosome: [1, 0, 1, 1, 1, 1, 0, 0]
Fitness Value: 11.0
Chromosome: [0, 0, 1, 0, 0, 0, 0, 1]
Fitness Value: 0
Chromosome: [1, 1, 1, 0, 0, 0, 0, 1]
Fitness Value: 9.0
Chromosome: [1, 0, 1, 1, 0, 0, 0, 0]
Fitness Value: 8.0
Chromosome: [0, 1, 1, 0, 0, 0, 0, 0]
Fitness Value: 4.0
Chromosome: [0, 0, 0, 0, 0, 1, 1, 1]
Fitness Value: 0
Chromosome: [0, 1, 1, 1, 0, 0, 0, 1]
Fitness Value: 10.0
Chromosome: [1, 0, 0, 0, 1, 1, 0, 0]
Fitness Value: 0
Chromosome: [1, 1, 1, 1, 1, 1, 1, 1]
Fitness Value: 17.0
Chromosome: [1, 0, 0, 1, 0, 1, 1, 1]
Fitness Value: 11.0
Chromosome: [1, 0, 1, 0, 0, 0, 0, 1]
Fitness Value: 0
Chromosome: [0, 1, 1, 1, 1, 1, 0, 0]
Fitness Value: 10.0
Chromosome: [1, 0, 1, 0, 0, 0, 0, 1]
Fitness Value: 0
Ch

In [10]:
# Dummy function to test 
# 1. Selection Roulette Wheel
# 2. Single Point Crossover
# 3.

def selection_crossover_mutation():
    pop = population(25)

    # Calculate fitness values for each chromosome
    fitness_values = [fitness_function(chromosome) for chromosome in pop]
    
    # Select parents
    parent1, parent2 = selection_roulette_wheel(pop, fitness_values)
    
    # Print the selected parents
    print("Parent 1:", parent1)
    print("Parent 2:", parent2)    
    
    # Crossover
    child1, child2 = crossover_two_point(parent1, parent2)

    # Print the offspring
    print()
    print("Child 1:", child1)
    print("Child 2:", child2)
    
    # Apply mutation
    mutated_child1 = mutation_inversion(child1)
    mutated_child2 = mutation_inversion(child2)

    # Print the mutated offspring after inversion mutation
    print()
    print("Mutated Child 1 (Inversion):", mutated_child1)
    print("Mutated Child 2 (Inversion):", mutated_child2)
    
    # Apply bit flipping mutation
    mutation_rate = 0.1
    mutated_child1 = mutation_bit_flip(child1, mutation_rate)
    mutated_child2 = mutation_bit_flip(child2, mutation_rate)

    # Print the mutated offspring after bit flipping mutation
    print()
    print("Mutated Child 1 (Bit Flip):", mutated_child1)
    print("Mutated Child 2 (Bit Flip):", mutated_child2)
    
selection_crossover_mutation()

Parent 1: [1, 1, 0, 0, 1, 0, 1, 0]
Parent 2: [0, 1, 1, 1, 1, 1, 0, 1]

Child 1: [0, 1, 1, 0, 1, 0, 1, 0]
Child 2: [1, 1, 0, 1, 1, 1, 0, 1]

Mutated Child 1 (Inversion): [0, 1, 1, 0, 1, 0, 1, 0]
Mutated Child 2 (Inversion): [1, 1, 0, 0, 1, 1, 1, 1]

Mutated Child 1 (Bit Flip): [0, 1, 0, 0, 1, 0, 0, 1]
Mutated Child 2 (Bit Flip): [1, 1, 0, 1, 1, 1, 0, 1]


In [11]:
def main():
    # Generate Population of 25 chromosomes
    pop = population(25) 
        
    # Calculate fitness values for each chromosome
    fitness_values = [fitness_function(chromosome) for chromosome in pop]


## Summary


# Question 2



# Question 3
